In [106]:
import pandas as pd
from kafka import KafkaConsumer, KafkaProducer
from time import sleep 
from json import dumps 
import json

In [107]:
producer = KafkaProducer(bootstrap_servers=['okore-joel:9092'],
                         value_serializer=lambda x:
                         dumps(x).encode('utf-8'))

In [108]:
data = pd.read_csv('credit_card_transactions.csv') # Load your data

fraud = data[data['is_fraud'] == 1]
non_fraud = data[data['is_fraud'] == 0]

# Downsample majority class (non_fraud) to match minority class (fraud)
non_fraud_sample = non_fraud.sample(n=len(fraud), random_state=42)

# Combine both minority class and downsampled majority class
df_balanced = pd.concat([fraud, non_fraud_sample])


In [109]:
df = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [110]:
from datetime import datetime
import time

for index, row in df.drop('is_fraud', axis=1).iterrows():
    # Add a timestamp to the transaction
    row_dict = row.to_dict()
    row_dict['sent_timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
    
    # Send transaction to Kafka topic
    producer.send('demo_test', value=row_dict)
    
    if df.iloc[index]['is_fraud'] == 1:
        print(f"[{row_dict['sent_timestamp']}] Transaction for {df.iloc[index]['first']} {df.iloc[index]['last']} || Transaction: {df.iloc[index]['trans_num']} (accept)")
    else:
        print(f"[{row_dict['sent_timestamp']}] Transaction for {df.iloc[index]['first']} {df.iloc[index]['last']} || Transaction: {df.iloc[index]['trans_num']} (decline)")
    
    time.sleep(1)  # Simulate real-time streaming by adding a delay
producer.flush()  # Ensure all data is sent

[2024-10-14 15:41:14.888515] Transaction for Anthony Allen || Transaction: c48f2efccb689f2daef9ebdb03e1edec (accept)
[2024-10-14 15:41:15.890362] Transaction for Daniel Boyd || Transaction: 68f2b98cba5c4ae6b7ff3f701d0cdc69 (accept)
[2024-10-14 15:41:16.892131] Transaction for Micheal Walters || Transaction: b832170811fa7ce7ec76d9af27adecd2 (decline)
[2024-10-14 15:41:17.894174] Transaction for Jeffrey Munoz || Transaction: 7bbb699e093e3fb7d16c96093e95d66f (decline)
[2024-10-14 15:41:18.896386] Transaction for Joanna Hudson || Transaction: 5666921b2c9a994021da1bdbb9aea908 (accept)
[2024-10-14 15:41:19.898455] Transaction for Christine Harris || Transaction: 72eefdc7ab123dfc9a4adee5aeae9e11 (accept)
[2024-10-14 15:41:20.900579] Transaction for Derek Jones || Transaction: ca0183f90d413fbc28e0a30912bfc386 (accept)
[2024-10-14 15:41:21.902895] Transaction for Monique Martin || Transaction: 99eff11c4cf99723aa5948d4953fa231 (decline)
[2024-10-14 15:41:22.905066] Transaction for Mary Mcintyre 

KeyboardInterrupt: 

In [ ]:
producer.close()

In [86]:
sample

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode
559238,559238,2019-08-26 09:07:39,3590736522064285,fraud_Zboncak Ltd,food_dining,116.96,Kimberly,Gonzalez,F,72966 Shannon Pass Apt. 391,...,-92.4828,4074,"Scientist, audiological",1975-12-20,d80702113c8924b2a3206e07fc903bd0,1345972059,35.150273,-92.062596,1,72045.0
987687,987687,2020-02-07 02:02:00,3598304579370089,fraud_Koepp-Witting,grocery_pos,306.43,Vincent,Flowers,M,0189 Emily Prairie,...,-97.1714,409656,Heritage manager,1985-12-27,39247af25d0a8ab5f0ec4889fa7ee323,1360202520,37.879840,-96.462024,1,NaN
551313,551313,2019-08-24 05:33:05,4168696938629925,"fraud_Heller, Gutmann and Zieme",grocery_pos,306.93,Kevin,Hawkins,M,92811 Jared Spur Apt. 345,...,-73.4030,19803,Police officer,1998-12-29,b5d77d0e3f0bfeeaa5c4cf9d0389c232,1345786385,41.091428,-74.097013,1,10952.0
588057,588057,2019-09-06 23:07:04,3551217896304745,fraud_Schumm PLC,shopping_net,6.70,Sara,Harris,F,597 Jenny Ford Apt. 543,...,-74.4905,6120,Chartered loss adjuster,1976-10-09,1165cff496001b3550ec0c9776d421d4,1346972824,43.439695,-73.605987,0,12844.0
1173129,1173129,2020-05-04 18:52:50,375848982312810,"fraud_Ledner, Hartmann and Feest",health_fitness,10.14,Dale,Smith,M,25036 Contreras Turnpike Suite 270,...,-97.9903,258,Web designer,1943-12-15,3879813e90ed30c5d3319964d9ae0936,1367693570,31.451097,-97.303447,0,76657.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419577,419577,2019-07-07 23:01:02,342360239917375,"fraud_Greenholt, O'Hara and Balistreri",shopping_net,1062.13,Gabriella,Jordan,F,1299 Joshua Plains,...,-74.3648,1304,Jewellery designer,1970-06-09,e3421507484596c2378e10c874359eb8,1341702062,42.902316,-74.324727,1,12016.0
721620,721620,2019-11-04 04:09:56,2256234701263057,"fraud_Robel, Cummerata and Prosacco",gas_transport,64.29,Christopher,Grimes,M,39631 Dunn Radial,...,-72.1334,2971,Charity fundraiser,1948-04-11,ce06a80614847ba36ad4d3930bfffda1,1352002196,43.032484,-72.913999,0,5360.0
297524,297524,2019-05-25 23:12:10,371226440126102,"fraud_Champlin, Rolfson and Connelly",travel,9.52,Stacy,Lambert,F,85430 Julie Trafficway Suite 792,...,-89.2463,1810,Race relations officer,1974-12-24,a926c702a9196de38c0211e50cfba05a,1337987530,39.855049,-89.608769,1,62684.0
865233,865233,2019-12-18 21:42:33,4601636330889,fraud_Schmidt and Sons,shopping_net,790.34,Mark,Rivera,M,383 Contreras Hills Suite 658,...,-95.2825,194500,Tax adviser,1990-12-18,56d83998c2bca807ab4a26f2a252bbe8,1355866953,29.122295,-95.928656,1,77468.0
